# Prototyping Dialogue Games
This notebook is following the tutorial steps from **docs/howto_prototype_games.ipynb** with some more detailed description of each step (when necessary)

In [ ]:
import sys
from string import Template
from backends import ModelSpec, Model, get_model_for, load_model_registry


load_model_registry()  # load default model registry from backends folder
THIS_MODEL = 'gpt-4o-mini-2024-07-18'
llm: Model = get_model_for(THIS_MODEL)

llm.set_gen_args(temperature=0.0, max_tokens=100)
llm.generate_response